In [24]:
! mkdir ../data/labels
! mkdir ../data/images
! cp ../data/raw/2024-GameTileNet-main/DataAndAnnotations/AnnotationsAndResults/vlm_author_labeled_tiles/* ../data/labels/
! rm ../data/labels/vlm_caption_results_bicubic.json
! rsync -a --exclude='*complete*' ../data/raw/2024-GameTileNet-main/DataAndAnnotations/Assets/* ../data/images/

mkdir: nie można utworzyć katalogu „../data/labels”: Plik istnieje


mkdir: nie można utworzyć katalogu „../data/images”: Plik istnieje


In [25]:
import json
from pathlib import Path
import re

new_base = "../data/images"

def fix_path(p):
    p = re.sub(r"[\\/]+", "/", p) # normalize slashes
    parts = p.split("/")
    tail = "/".join(parts[-2:])
    return f"{new_base}/{tail}"

def generate_caption_from_filename(filename):
    name = Path(filename).stem                       # without extension
    name = re.sub(r"[0-9]+", "", name)               # remove digits
    name = re.sub(r"[^A-Za-z]", " ", name)           # replace non-letters with spaces
    name = re.sub(r"([a-z])([A-Z])", r"\1 \2", name) # split camel case
    return f"{name.lower()}"

def has_repeated_word(caption):
    "Checks if there are any repeated words in the caption."
    words = caption.split()
    return len(words) != len(set(words))

total_records = 0

for json_file in Path("../data/labels").rglob("*.json"):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    for item in data:
        if "image" in item:
            item["image"] = fix_path(item["image"])
        
        if "caption" in item:
            if has_repeated_word(item["caption"]):
                item["caption"] = generate_caption_from_filename(item["image"])

    total_records += len(data)

    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"Updated: {json_file}")

print(f"Total records processed: {total_records}")

Updated: ../data/labels/vlm_caption_results_author_bicubic.json
Updated: ../data/labels/vlm_caption_results_author_swinir_x4.json
Updated: ../data/labels/vlm_caption_results_author_realesrgan_x4.json
Updated: ../data/labels/vlm_caption_results_author_sd_fidelity.json
Updated: ../data/labels/vlm_caption_results_author_original.json
Updated: ../data/labels/vlm_caption_results_author_sd_img2img.json
Total records processed: 11379
